# Exercises


<!-- examples on model4 with 1, x, x^2 etc. Show that N=2 recovers -->
<!-- the exact solution -->

<!-- heat conduction in the ground with radioactivity, need good  model, not -->
<!-- just the old one -->
<!-- string with load -->
<!-- hanging cable, see ideas/5620 articles about that (nonlinear) -->



<!-- --- begin exercise --- -->

## Exercise 1: Refactor functions into a more general class
<div id="fem:deq:exer:BVP1D:class"></div>

The section [Simple model problems and their solutions](#fem:deq:1D:models:simple) lists three functions for
computing the analytical solution of some simple model problems. There
is quite some repetitive code, suggesting that the functions can
benefit from being refactored into a class hierarchy, where the super
class solves $-(a(x)u'(x))'=f(x)$ and where subclasses define the
equations for the boundary conditions in a model. Make a method for
returning the residual in the differential equation and the boundary
conditions when the solution is inserted in these equations. Create a
test function that verifies that all three residuals vanish for each
of the model problems in the section [Simple model problems and their solutions](#fem:deq:1D:models:simple).  Also
make a method that returns the solution either as `sympy` expression
or as a string in LaTeX format.  Add a fourth subclass for the problem
$-(au')'=f$ with a Robin boundary condition:

$$
u(0)=0,\quad -u'(L) = C(u - D){\thinspace .}
$$

Demonstrate the use of this subclass for the case $f=0$ and $a=\sqrt{1+x}$.


<!-- --- begin solution of exercise --- -->
**Solution.**
This is an exercise in software engineering.
The model-specific information is related to the boundary
conditions only. We can then let the super class take care of the
differential equation and the solution process, while subclasses
provide a method `get_bc` to
return the symbolic expressions for the boundary equations.

The super class may be coded as
shown below.

In [18]:
import sympy as sym
x, L, C, D, c_0, c_1, = sym.symbols('x L C D c_0 c_1')

class TwoPtBoundaryValueProblem(object):
    """
    Solve -(a*u')' = f(x) with boundary conditions
    specified in subclasses (method get_bc).
    a and f must be sympy expressions of x.
    """
    def __init__(self, f, a=1, L=L, C=C, D=D):
        """Default values for L, C, D are symbols."""
        self.f = f
        self.a = a
        self.L = L
        self.C = C
        self.D = D

        # Integrate twice
        u_x = - sym.integrate(f, (x, 0, x)) + c_0
        u = sym.integrate(u_x/a, (x, 0, x)) + c_1
        # Set up 2 equations from the 2 boundary conditions and solve
        # with respect to the integration constants c_0, c_1
        eq = self.get_bc(u)
        eq = [sym.simplify(eq_) for eq_ in eq]
        print(('BC eq:', eq))
        self.u = self.apply_bc(eq, u)

    def apply_bc(self, eq, u):
        # Solve BC eqs respect to the integration constants
        r = sym.solve(eq, [c_0, c_1])
        # Substitute the integration constants in the solution
        u = u.subs(c_0, r[c_0]).subs(c_1, r[c_1])
        u = sym.simplify(sym.expand(u))
        return u

    def get_solution(self, latex=False):
        return sym.latex(self.u, mode='plain') if latex else self.u

    def get_residuals(self):
        """Return the residuals in the equation and BCs."""
        R_eq = sym.diff(sym.diff(self.u, x)*self.a, x) + self.f
        R_0, R_L = self.get_bc(self.u)
        residuals = [sym.simplify(R) for R in (R_eq, R_0, R_L)]
        return residuals

    def get_bc(self, u):
        raise NotImplementedError(
            'class %s has not implemented get_bc' %
            self.__class__.__name__)

The various subclasses deal with the boundary conditions of the
various model problems:

In [19]:
class Model1(TwoPtBoundaryValueProblem):
    """u(0)=0, u(L)=D."""
    def get_bc(self, u):
        return [u.subs(x, 0)-0,               # x=0 condition
                u.subs(x, self.L) - self.D]   # x=L condition

class Model2(TwoPtBoundaryValueProblem):
    """u'(0)=C, u(L)=D."""
    def get_bc(self, u):
        return [sym.diff(u,x).subs(x, 0) - self.C, # x=0 cond.
                u.subs(x, self.L) - self.D]        # x=L cond.

class Model3(TwoPtBoundaryValueProblem):
    """u(0)=C, u(L)=D."""
    def get_bc(self, u):
        return [u.subs(x, 0) - self.C,
                u.subs(x, self.L) - self.D]

A suitable test function gets quite compact:

In [20]:
def test_TwoPtBoundaryValueProblem():
    f = 2
    model = Model1(f)
    print(('Model 1, u:', model.get_solution()))
    for R in model.get_residuals():
        assert R == 0

    f = x
    model = Model2(f)
    print(('Model 2, u:', model.get_solution()))
    for R in model.get_residuals():
        assert R == 0

    f = 0
    a = 1 + x**2
    model = Model3(f, a=a)
    print(('Model 3, u:', model.get_solution()))
    for R in model.get_residuals():
        assert R == 0

The fourth model is just about defining the boundary conditions as equations:

In [21]:
class Model4(TwoPtBoundaryValueProblem):
    """u(0)=0, -u'(L)=C*(u-D)."""
    def get_bc(self, u):
        return [u.subs(x, 0) - 0,
                -sym.diff(u, x).subs(x, self.L) -
                self.C*(u.subs(x, self.L) - self.D)]

A demo function goes like

In [22]:
def demo_Model4():
    f = 0
    model = Model4(f, a=sym.sqrt(1+x))
    print(('Model 4, u:', model.get_solution()))

The printout shows that the solution is

$$
u(x) = \frac{2CD\sqrt{1+L}(\sqrt{1+x}-1}{2C\sqrt{1+L} + 2C+1}{\thinspace .}
$$

<!-- --- end solution of exercise --- -->
Filename: `uxx_f_sympy_class`.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 2: Compute the deflection of a cable with sine functions
<div id="fem:deq:exer:tension:cable"></div>

A hanging cable of length $L$
with significant tension $T$ has a deflection $w(x)$
governed by

$$
T w''(x) = \ell(x),
$$

where $\ell(x)$ the vertical load per unit length.
The cable is fixed at $x=0$ and $x=L$ so the boundary conditions become
$w(0)=w(L)=0$. The deflection $w$ is positive upwards, and $\ell$ is
positive when it acts downwards.

If we assume a constant load $\ell(x)=\hbox{const}$,
the solution is expected to be symmetric around $x=L/2$. For a function
$w(x)$ that is symmetric around some point $x_0$, it means that
$w(x_0-h) = w(x_0+h)$, and then $w'(x_0)=\lim_{h\rightarrow 0}(w(x_0+h)-
w(x_0-h))/(2h)=0$. We can therefore utilize symmetry to halve the domain.
We then seek $w(x)$ in $[0,L/2]$ with boundary conditions $w(0)=0$ and
$w'(L/2)=0$.

The problem can be scaled by introducing dimensionless  variables,

$$
\bar x = \frac{x}{L/2},\quad \bar u = \frac{w}{w_c},
$$

where $w_c$ is a characteristic size of $w$.
Inserted in the problem for $w$,

$$
\frac{4Tw_c}{L^{2}}\frac{d^2\bar u}{d\bar x^2} = \ell\ (= \hbox{const}){\thinspace .}
$$

A desire is to have $u$ and its derivatives about unity, so
choosing $w_c$ such that $|d^2\bar u/d\bar x^2|=1$ is an idea.
Then $w_c=\frac{1}{4}\ell L^2/T$, and the problem for the scaled vertical
deflection $u$ becomes

$$
u'' = 1,\quad x\in (0,1),\quad u(0)=0,\ u'(1)=0{\thinspace .}
$$

Observe that there are no physical parameters in this scaled problem.
From now on we have for convenience
renamed $x$ to be the scaled quantity $\bar x$.


**a)**
Find the exact solution for the deflection $u$.


<!-- --- begin solution of exercise --- -->
**Solution.**
[Exercise 1: Refactor functions into a more general class](#fem:deq:exer:BVP1D:class) or
the section [Simple model problems and their solutions](#fem:deq:1D:models:simple) features tools for finding
the analytical solution of this differential equation.
The present
model problem is close to model 2
in the section [Simple model problems and their solutions](#fem:deq:1D:models:simple). We can modify the `model2`
function:

In [23]:
def model():
    """Solve u'' = -1, u(0)=0, u'(1)=0."""
    import sympy as sym
    x, c_0, c_1, = sym.symbols('x c_0 c_1')
    u_x = sym.integrate(1, (x, 0, x)) + c_0
    u = sym.integrate(u_x, (x, 0, x)) + c_1
    r = sym.solve([u.subs(x,0) - 0,
                   sym.diff(u,x).subs(x, 1) - 0],
                  [c_0, c_1])
    u = u.subs(c_0, r[c_0]).subs(c_1, r[c_1])
    u = sym.simplify(sym.expand(u))
    return u

The solution becomes

$$
u(x) = \frac{1}{2}x(x-2){\thinspace .}
$$

Plotting $u(x)$ shows that $|u|\in [0,\frac{1}{2}]$ which is compatible with
the aim of the scaling, i.e., to have $u$ of size *about* unity (at least
not very small or very large).

<!-- --- end solution of exercise --- -->

**b)**
A possible function space is spanned by ${\psi}_i=\sin ((2i+1)\pi x/2)$,
$i=0,\ldots,N$. These functions
fulfill the necessary condition ${\psi}_i(0)=0$,
but they also fulfill ${\psi}_i'(1)=0$ such that both boundary
conditions are fulfilled by the expansion $u=\sum_jc_j{\varphi}_j$.

Use a Galerkin and a least squares method to find the coefficients
$c_j$ in $u(x)=\sum_j c_j{\psi}_j$. Find how fast the coefficients
decrease in magnitude by looking at $c_j/c_{j-1}$.
Find the error in the maximum deflection at $x=1$ when only one
basis function is used ($N=0$).

<!-- --- begin hint in exercise --- -->

**Hint.**
In this case, where the basis functions and their derivatives are
orthogonal, it is easiest to set up the calculations by hand and
use `sympy` to help out with the integrals.

<!-- --- end hint in exercise --- -->


<!-- --- begin solution of exercise --- -->
**Solution.**
With $u=\sum_{j=0}^Nc_j{\psi}_j(x)$ the residual becomes

$$
R = 1 - u'' = 1 +\sum_{j=0}^Nc_j{\psi}_j''(x) =
1 + \sum_{j=0}^Nc_j(2j+1)^2\frac{\pi^2}{4}\sin((2j+1)\frac{\pi x}{2}){\thinspace .}
$$

**Least squares method.**
The minimization of $\int_0^1R^2dx$ leads to the equations

$$
(R,\frac{\partial R}{\partial c_i})=0,\quad i=0,\ldots,N{\thinspace .}
$$

We find that

$$
\frac{\partial R}{\partial c_i} =
(2i+1)^2\frac{\pi^2}{4}\sin((2i+1)\frac{\pi x}{2}),
$$

so the governing equations become

$$
(1+\sum_{j=0}^Nc_j(2j+1)^2\frac{\pi^2}{4}\sin((2j+1)\frac{\pi x}{2}),
(2i+1)^2\frac{\pi^2}{4}\sin((2i+1)\frac{\pi x}{2}) = 0{\thinspace .}
$$

By linearity of the inner product (or integral) this expression can
be reordered to

$$
\begin{align*}
\sum_{j=0}^Nc_j((2j+1)^2\frac{\pi^2}{4}\sin((2j+1)\frac{\pi x}{2}), &
(2i+1)^2\frac{\pi^2}{4}\sin((2i+1)\frac{\pi x}{2}) = \\ 
& -(1, (2i+1)^2\frac{\pi^2}{4}\sin((2i+1)\frac{\pi x}{2})),
\end{align*}
$$

which is nothing but a linear system

$$
\sum_{j=0}^N A_{i,j}c_j = b_i,\quad i=0,\ldots,N,
$$

with

$$
\begin{align*}
A_{i,j} &= (2j+1)^4\frac{\pi^4}{16}\int_0^1
\sin((2j+1)\frac{\pi x}{2})\sin((2i+1)\frac{\pi x}{2})dx,\\ 
b_i &= -(2i+1)^2\frac{\pi^2}{4}\int_0^1 \sin((2i+1)\frac{\pi x}{2})dx
\end{align*}
$$

Orthogonality of the sine functions $\sin (k\pi x/2)$ on $[0,1]$
for integer $k$ implies that
$A_{i,j}=0$ for $i\neq j$, and $A_{i,i}$ can be
computed by `sympy`:

In [24]:
from sympy import *
i = symbols('i', integer=True)
x = symbols('x', real=True)
integrate(sin(i*pi*x/2)**2, (x, 0, 1))

Therefore,

$$
A_{i,j} = \left\lbrace\begin{array}{ll}
0,& i\neq j\\ 
\frac{1}{2} (2i+1)^4\frac{\pi^4}{16}, & i = j
\end{array}\right.
$$

The right-hand side can also be computed by `sympy`:

In [25]:
integrate(sin((2*i+1)*pi*x/2), (x, 0, 1))

One should always be skeptical to symbolic software and integration of
periodic functions like the sine and cosine since the answers can be
too simplistic (see subexercise d!).
A general test is to perform numerical integration with
lots of sampling points to (partially) verify the symbolic formula.
Here is an application of the midpoint rule:

In [26]:
def midpoint_rule(f, M=100000):
    """Integrate f(x) over [0,1] using M intervals."""
    from numpy import sum, linspace
    dx = 1.0/M                       # interval length
    x = linspace(dx/2, 1-dx/2, M)    # integration points
    return dx*sum(f(x))

def check_integral_b():
    from numpy import pi, sin
    for i in range(12):
        exact = 2/(pi*(2*i+1))
        numerical = midpoint_rule(
            f=lambda x: sin((2*i+1)*pi*x/2))
        print((i, abs(exact - numerical)))

The output shows that the difference between numerical and exact
integration is about $10^{-11}$, which is "small" (and gets smaller
by just increasing `M`). This result brings evidence that the
`sympy` answer is correct.
Alternatively, in this simple case, we can easily calculate the anti-derivative.
It goes like

$$
-\frac{2}{\pi(2i+1)}\cos((2k+1)\frac{\pi x}{2}),
$$

and for $x=1$ we get
$\cos\frac{\pi}{2}$, $\cos 3\frac{\pi}{2}$, $\cos 5\frac{\pi}{2}$,
and so on, which all evaluates to zero, and since the cosine is 1 for $x=0$,
the formula found by `sympy` is correct.

We then get

$$
b_i = -(2i+1)^2\frac{\pi^2}{4}\frac{2}{\pi(2i+1)} = -\frac{1}{2} (2i+1)\pi,
$$

and consequently,

$$
c_i = \frac{b_i}{A_{i,i}} = -\frac{\frac{1}{2} (2i+1)\pi}{\frac{1}{2} (2i+1)^4}\frac{\pi^4}{16} = -\frac{16}{\pi^3(2i+1)^3}{\thinspace .}
$$

**Galerkin's method.**
The Galerkin method applied to this problem
starts with

$$
(u'',v) = (1,v)\quad \forall v\in V,
$$

and the requirement that $v(0)=0$ since $u(0)=0$.
Integration by parts and using $u'(1)=0$ and $v(0)=0$ makes the boundary
term vanish, and the variational form becomes

$$
(u',v') = -(1,v) \quad \forall v\in V{\thinspace .}
$$

Inserting $u=\sum_{j=0}^Nc_j{\psi}_j(x)$ and $v={\psi}_i$ leads to

$$
\sum_{j=0}^N ({\psi}_j', {\psi}_i')c_j = (1,{\psi}_i),\quad i=0,\ldots,N{\thinspace .}
$$

With ${\psi}_i=\sin((2i+1)\frac{\pi x}{2})$ the matrix entries become

$$
A_{i,j} =  (2i+1)(2j+1)\frac{\pi^2}{4}\int_0^1 \cos((2i+1)\frac{\pi x}{2})
\cos((2j+1)\frac{\pi x}{2})dx{\thinspace .}
$$

Orthogonality of the cosine functions implies $A_{i,j}=0$ for
$i\neq j$, and $A_{i,i}$ is computed by integrating the square
of the cosine function,

In [27]:
integrate(cos((k+1)*pi*x/2)**2, (x, 0, 1))

Now,

$$
A_{i,i} = (2i+1)^2\frac{\pi^2}{4}\frac{1}{2} = \frac{1}{8}(2i+1)^2 \pi^2{\thinspace .}
$$

The right-hand side has almost the same integral as in the
least squares case,

$$
b_i = -\int_0^1 \sin((2i+1)\frac{\pi x}{2})dx = -\frac{2}{\pi (2i+1)}{\thinspace .}
$$

Consequently,

$$
c_i = \frac{b_i}{A_{i,i}} = -\frac{16}{\pi^3(2i+1)^3},
$$

which is the same result as we obtained in the least squares method.


**Decay of coefficients.**
The coefficients decay,

$$
\frac{c_i}{c_{i+1}} = \left(\frac{2i+3}{2i+1}\right)^3 > 0{\thinspace .}
$$

The decay is most pronounced for the first terms:

In [28]:
for i in range(10):
  print((float(2*i+3)/(2*i+1))**3)

**Error in one-term solution.**
Keeping just one term ($N=0$) means that

$$
u(x) = -\frac{16}{\pi^3}\sin(\frac{\pi x}{2}){\thinspace .}
$$

The maximum deflection at $x=1$ becomes $-16\pi^{-3}=-0.5160$, to be compared
with the exact value $-\frac{1}{2}$. The error is 3.2 percent.

<!-- --- end solution of exercise --- -->

**c)**
Visualize the solutions in b) for $N=0,1,20$.


<!-- --- begin solution of exercise --- -->
**Solution.**
First we need a function to compute the approximate $u$ in this case:

In [29]:
def sine_sum(x, N):
    s = 0
    from numpy import pi, sin, zeros
    u = [] # u[k] is the sum i=0,...,k
    k = 0
    for i in range(N+1):
        s += - 16.0/((2*i+1)**3*pi**3)*sin((2*i+1)*pi*x/2)
        u.append(s.copy())  # important with copy!
    return u

Note the need to append `s.copy()`: doing just `u.append(s)` will
make, e.g., `u[0]` a reference to `s`, which at the end of the
loop is an array corresponding to the maximum $i$ value.

We also need a function that can create an appropriate plot:

In [30]:
def plot_sine_sum():
    from numpy import linspace
    x = linspace(0, 1, 501)  # coordinates for plot
    u = sine_sum(x, N=10)
    u_e = 0.5*x*(x-2)
    N_values = 0, 1, 10
    for k in N_values:
        plt.plot(x, u[k])
    plt.plot(x, u_e)
    plt.legend(['N=%d' % k for k in N_values] + ['exact'],
               loc='upper right')
    plt.xlabel('$x$');  plt.ylabel('$u$')
    plt.savefig('tmpc.png'); plt.savefig('tmpc.pdf')

The plot shows that the solution for $N=0$ has a slight deviation from the
exact curve, but even $N=1$ catches up visually with the exact solution (!).

<!-- dom:FIGURE: [fig/cable_sin_c.png, width=500 frac=0.8] -->
<!-- begin figure -->

<p></p>
<img src="fig/cable_sin_c.png" width=500>

<!-- end figure -->


<!-- --- end solution of exercise --- -->

**d)**
The functions in b) were selected such that they fulfill the
condition ${\psi}'(1)=0$. However, in the Galerkin method, where we
integrate by parts, the condition $u'(1)=0$ is incorporated in the
variational form. This leads to the idea of just choosing a simpler
basis, namely "all" sine functions ${\psi}_i = \sin((i+1)\frac{\pi x}{2})$.
Will the method adjust the coefficient such that the additional
functions compared with those in b) get vanishing coefficients? Or
will the additional basis functions improve the solution?
Use Galerkin's method.


<!-- --- begin solution of exercise --- -->
**Solution.**
According to the calculations in b), the Galerkin method, with
${\psi}_i = \sin((i+1)\frac{\pi x}{2})$, leads to the almost the
same
matrix entries on the diagonal:

$$
\begin{align*}
A_{i,i} &= (i+1)(j+1)\frac{\pi^2}{4}\int_0^1 \cos((i+1)\frac{\pi x}{2})
\cos((j+1)\frac{\pi x}{2})dx\\ 
&= (i+1)^2\frac{\pi^2}{4}\frac{1}{2} = \frac{1}{8}(i+1)^2 \pi^2{\thinspace .}
\end{align*}
$$

The right-hand side becomes (as before)

$$
b_i = -\int_0^1 \sin((i+1)\frac{\pi x}{2})dx = -\frac{2}{\pi (i+1)}{\thinspace .}
$$

We may use `sympy` to integrate,

In [31]:
integrate(sin((i+1)*pi*x/2), (x, 0, 1))

As noted in b), let us be a bit skeptical to this answer and check it.
A quick check with numerical integration,

In [32]:
def check_integral_d_sympy_answer():
    from numpy import pi, sin
    for i in range(12):
        exact = 2/(pi*(i+1))
        numerical = midpoint_rule(
            f=lambda x: sin((i+1)*pi*x/2))
        print((i, abs(exact - numerical)))

gives the output

        0 6.54487575247e-12
        1 0.31830988621
        2 1.96350713466e-11
        3 0.159154943092
        4 3.27249061183e-11
        5 0.106103295473
        6 4.58150045679e-11
        7 0.0795774715459
        8 5.89047144395e-11
        9 0.0636619773677
        10 7.19949447281e-11
        11 0.0530516476973


It is clear that for $i$ odd, there are significant differences between
the `sympy` answer and the midpoint rule with high resolution!

We therefore need to do hand calculations to investigate this problem
further.
The anti-derivative is very easy to realize in this case:

$$
\begin{align*}
\int_0^1\sin ((i+1)\pi x/2)dx &= -\frac{2}{\pi(i+1)}(\cos((i+1)\frac{\pi}{2}) - \cos(0))\\ 
&= \frac{2}{\pi(i+1)}(1 - \cos((i+1)\frac{\pi}{2})){\thinspace .}
\end{align*}
$$

The value of the cosine expression depends on $i$, and the first values are

<table border="1">
<thead>
<tr><th align="center">$i=0$</th> <th align="center">$i=1$</th> <th align="center">$i=2$</th> <th align="center">$i=3$</th> </tr>
</thead>
<tbody>
<tr><td align="center">   0        </td> <td align="center">   -1       </td> <td align="center">   0        </td> <td align="center">   1        </td> </tr>
</tbody>
</table>
This pattern repeats and is the same for four consecutive values of $i$.
Hence, the integral is $2/(\pi (i+1))$ for even $i$ ($i=2k$ for
integer $k$, or equivalently: when $i\mbox{ mod } 2 = 0$). For $i=4k+1$, or
equivalently: when $(i-1)\mbox{ mod } 4 = 0$, the
integral is $4/(\pi(4k+1))$, while for $i=4k+3$, the integral vanishes.
This is a more complicated answer than what `sympy` provides!

We can check our new answers against numerical integration:

In [33]:
def check_integral_d():
    from numpy import pi, sin
    for i in range(24):
        if i % 2 == 0:
            exact = 2/(pi*(i+1))
        elif (i-1) % 4 == 0:
            exact = 2*2/(pi*(i+1))
        else:
            exact = 0
        numerical = midpoint_rule(
            f=lambda x: sin((i+1)*pi*x/2))
        print((i, abs(exact - numerical)))

The output now is around $10^{-10}$ and we take that as a sign that
our exact results are reliable.

**Carefully check symbolic computations!**

The example above shows how `sympy` can fail.
[Wolfram Alpha](http://wolframalpha.com) does a better job: writing
`integrate sin(k*x*pi/2) from 0 to 1` (use `k` instead of `i` since the latter
is the imaginary unit) returns the [result](http://www.wolframalpha.com/input/?i=integrate+sin%28k*x*pi%2F2%29+from+0+to+1) $4\sin^2(\pi k/4)/(\pi k)$,
which coincides with out result.

There are three general techniques
to verify a symbolic computation:

 * Use alternative software like Wolfram Alpha for comparison

 * Check that the result satisfies the problem to be solved

 * Make a high-resolution numerical approximation and compare

(The second technique is not so applicable here, since we work with
a definite integral, but one could compute the indefinite integral
instead, which is done correctly by `sympy`, and discuss values for
$x=1$.)



The final result for $c_i$ is now

$$
c_i = \frac{b_i}{A_{i,i}} = \left\lbrace\begin{array}{ll}
-\frac{16}{\pi^3(i+1)^3}, & i\hbox{ even, or } i \hbox{ mod } 2 = 0\\ 
-\frac{32}{\pi^3(i+1)^3}, & (i-1)\hbox{ mod } 4 = 0,\\ 
0, & (i+1)\hbox{ mod } 4 = 0
\end{array}\right.
$$

We recognize that for $i$ even, say $i=2k$ for integer $k$, we
have exactly the same result as in b):

$$
-\sum_k \frac{16}{\pi^3(2k+1)^3}\sin((2k+1)x\frac{\pi x}{2}),
$$

but we get an additional set of terms for $i=4k+1$,

<!-- Equation labels as ordinary links -->
<div id="fem:deq:exer:tension:cable:badterms"></div>

$$
\begin{equation}
-\sum_k \frac{32}{\pi^3(i+1)^3}\sin((4k+1)x\frac{\pi x}{2}){\thinspace .}
\label{fem:deq:exer:tension:cable:badterms} \tag{63}
\end{equation}
$$

We can modify the software from c) to compute the approximate $u$
with the present set of basis functions and coefficients:

In [34]:
def sine_sum_d(x, N):
    s = 0
    from numpy import pi, sin, zeros
    u = []  # u[k] is the sum i=0,...,k
    k = 0
    for i in range(N+1):
        if i % 2 == 0:       # even i
            s +=   - 16.0/((i+1)**3*pi**3)*sin((i+1)*pi*x/2)
        elif (i-1) % 4 == 0:   # 1, 5, 9, 13, 17
            s += - 2*16.0/((i+1)**3*pi**3)*sin((i+1)*pi*x/2)
        else:
            s += 0
        u.append(s.copy())
    return u

def plot_sine_sum_d():
    from numpy import linspace
    x = linspace(0, 1, 501)  # coordinates for plot
    u = sine_sum_d(x, N=20)
    u_e = 0.5*x*(x-2)
    N_values = 0, 1, 2, 3, 20
    for k in N_values:
        plt.plot(x, u[k])
    plt.plot(x, u_e)
    plt.legend(['N=%d' % k for k in N_values] + ['exact'],
               loc='upper right')
    plt.xlabel('$x$');  plt.ylabel('$u$')
    #plt.axis([0.9, 1, -0.52, -0.49])
    plt.savefig('tmpd.png'); plt.savefig('tmpd.pdf')

The approximations for $N=0,1,3,20$ appear below.

<!-- dom:FIGURE: [fig/cable_sin_d.png, width=500 frac=0.8] -->
<!-- begin figure -->

<p></p>
<img src="fig/cable_sin_d.png" width=500>

<!-- end figure -->


While the approximation for $N=0$ coincides with the one in b), we
see that $N=1$ and higher values of $N$ lead to a clearly wrong curve.
This strange feature has to be investigated!

Let us start by plotting the basis functions for $i=0,1,\ldots,7$:

<!-- dom:FIGURE: [fig/sinix_int.png, width=800 frac=1] -->
<!-- begin figure -->

<p></p>
<img src="fig/sinix_int.png" width=800>

<!-- end figure -->


We observe from the figure that all the basis functions corresponding to
even $i$ are symmetric around $x=1$, which is an important property of
the solution. The functions for odd $i$ are anti-symmetric. However,
for $i=3,7,11,\ldots$ the basis function has an integer number of
periods on $[0,1]$ so the integral becomes zero, $c_i=0$, and
consequently
there is no effect from these functions. The functions corresponding
to $i=1,5,9,13,\ldots$ are anti-symmetric around $x=1$ with nonzero
coefficients. The derivative of an anti-symmetric function at the point
of anti-symmetry is unity in size. Since the derivatives of all the
basis functions corresponding to even $i$ vanish at $x=1$, the
extra terms ($i=1,5,9,13,\ldots$) in ([63](#fem:deq:exer:tension:cable:badterms))
have a nonzero derivative, resulting in $u'(1)\neq 0$. That is,
these terms destroy the solution!

But we computed $c_i$ by a Galerkin method, which is equivalent to a
least squares method, which gives us the "best" approximation possible?
That is true, but it is the best approximation in the chosen space $V$.
The problem is that we have populated (or rather polluted) the space
$V$ with some basis functions that have a wrong mathematical property: they
are anti-symmetric around $x=1$.

<!-- --- end solution of exercise --- -->

**e)**
Now we drop the symmetry condition at $x=1$ and extend the domain to
$[0,2]$ such that it covers the entire (scaled) physical cable. The
problem now reads

$$
u'' = 1,\quad x\in (0,2),\quad u(0)=u(2)=0{\thinspace .}
$$

This time we need basis functions that are zero at $x=0$ and $x=2$.
The set $\sin((i+1)\frac{\pi x}{2})$ from d) is a candidate since
they vanish $x=2$ for any $i$. Compute the approximation in this case.
Why is this approximation without the problem that this set of
basis functions introduced in d)?


<!-- --- begin solution of exercise --- -->
**Solution.**
The formulas are almost the same as in d), only the integration domain
is different. Since the sine functions or orthogonal on $[0,1]$, they
are also orthogonal on $[0,2]$. Because

In [35]:
integrate(cos((i+1)*pi*x/2)**2, (x, 0, 2))

we get (in Galerkin's method)

$$
\begin{align*}
A_{i,i} &= (i+1)(j+1)\frac{\pi^2}{4}\int_0^2 \cos((i+1)\frac{\pi x}{2})
\cos((i+1)\frac{\pi x}{2})dx\\ 
& = (i+1)^2\frac{\pi^2}{4}{\thinspace .}
\end{align*}
$$

and

$$
b_i = -\int_0^2 \sin((i+1)\frac{\pi x}{2})dx = \frac{2}{\pi (i+1)}(\cos((i+1)\pi) - 1){\thinspace .}
$$

We have that $\cos((i+1)\pi = -1$ for $i$ even and
$\cos((i+1)\pi = 1$ for $i$ odd. That is,

$$
b_i =\left\lbrace\begin{array}{ll}
-\frac{4}{\pi (i+1)}, & i\hbox{ even }\\ 
0, & i\hbox{ odd }
\end{array}\right.
$$

The coefficients become

$$
c_i =\frac{b_i}{A_{i,i}} =\left\lbrace\begin{array}{ll}
-\frac{16}{\pi^3(i+1)^3}, & i\hbox{ even }\\ 
0, & i\hbox{ odd }
\end{array}\right.
$$

Introducing $i=2k$ and then switching from $k$ to $i$ as summation index
gives $c_i = -\frac{16}{\pi^3(2i+1)^3}$ and

$$
u(x) = -\sum_{i=0}^N \frac{16}{\pi^3(2i+1)^3}\sin((i+1)\frac{\pi x}{2}),
$$

which is the same expansion as in b).

The reason why the basis functions ${\psi}_i=\sin((i+1)\frac{\pi x}{2})$
work well in this case is that the problematic functions for $i=1,5,\ldots$
in d) now live on $[0,2]$ instead of $[0,1]$. On $[0,2]$ these functions
have an integer number of periods such that the integral from 0 to 2
becomes zero. These basis functions are therefore excluded from the
expansion since their coefficients vanish.
The lesson learned is that two equivalent boundary value
problems may make different demands to the basis functions.

<!-- --- end solution of exercise --- -->

<!-- BIG point: use polynomials, without integration by parts we cannot -->
<!-- handle the boundary condition. -->

Filename: `cable_sin`.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3: Compute the deflection of a cable with power functions
<div id="fem:deq:exer:tension:cable_xn"></div>


**a)**
Repeat [Exercise 2: Compute the deflection of a cable with sine functions](#fem:deq:exer:tension:cable) b), but work with
the space

$$
V = \hbox{span}\{x, x^2, x^3, x^4, \ldots\}{\thinspace .}
$$

Choose the dimension of $V$ to be 4 and observe that the exact solution
is recovered by the Galerkin method.

<!-- --- begin hint in exercise --- -->

**Hint.**
Use the `solver` function from `varform1D.py`.

<!-- --- end hint in exercise --- -->


<!-- --- begin solution of exercise --- -->
**Solution.**
The Galerkin formulation of $u''=1$, $u(0)=0$, $u'(1)=0$, reads

$$
(u',v') = -(1,v)\quad\forall v\in V,
$$

and the linear system becomes

$$
\sum_{j=}^N ({\psi}_i', {\psi}_j')c_j = -(1,{\psi}_i),\quad i=0,1,\ldots,N{\thinspace .}
$$

The `varform1D.solver` function needs a function specifying the integrands
on the left- and right-hand sides of the variational formulation.
Moreover, we must compute a dictionary of ${\psi}_i$ and ${\psi}_i'$.
The appropriate code becomes

In [36]:
from varform1D import solver
import sympy as sym
x, b = sym.symbols('x b')
f = 1

# Compute basis functions and their derivatives
N = 4
psi = {0: [x**(i+1) for i in range(N+1)]}
psi[1] = [sym.diff(psi_i, x) for psi_i in psi[0]]

# Galerkin

def integrand_lhs(psi, i, j):
    return psi[1][i]*psi[1][j]

def integrand_rhs(psi, i):
    return -f*psi[0][i]

Omega = [0, 1]

u, c = solver(integrand_lhs, integrand_rhs, psi, Omega,
              verbose=True, symbolic=True)
print(('Galerkin solution u:', sym.simplify(sym.expand(u))))

Running this code gives the output

        solution u: x*(x - 2)/2


which coincides with the exact solution ($c_3=c_4=0$).

<!-- --- end solution of exercise --- -->

**b)**
What happens if we use a least squares method for this problem with
the basis in a)?


<!-- --- begin solution of exercise --- -->
**Solution.**
The least squares formulation leads to

$$
(R,\frac{\partial R}{\partial c_i}=0,\quad i=0,\ldots,N,
$$

with

$$
R = 1 - u'' = 1 - \sum_jc_j{\psi}_j''{\thinspace .}
$$

We have

$$
\frac{\partial R}{\partial c_i} = {\psi}_i'',
$$

leading to the equations

$$
(1 + \sum_jc_j{\psi}_j'', {\psi}_i''),\quad i=0,\ldots,N,
$$

which is a linear system

$$
\sum_{j=0}^N({\psi}_j'',{\psi}_i'') = (-1,{\psi}_i''),\quad i=0,\ldots,N{\thinspace .}
$$

The fundamental problem with the basis in a) is that ${\psi}_0''=0$, so
if power functions of $x$ are wanted, we need to work with the basis
$V=\hbox{span}\{x^2, x^3,\ldots\}$. If we do so, we can easily modify
the code from a),

In [37]:
# Least squares
psi = {0: [x**(i+2) for i in range(N+1)]}
psi[1] = [sym.diff(psi_i, x) for psi_i in psi[0]]
psi[2] = [sym.diff(psi_i, x) for psi_i in psi[1]]

def integrand_lhs(psi, i, j):
    return psi[2][i]*psi[2][j]

def integrand_rhs(psi, i):
    return -f*psi[2][i]

Omega = [0, 1]

u, c = solver(integrand_lhs, integrand_rhs, psi, Omega,
              verbose=True, symbolic=True)
print(('solution u:', sym.simplify(sym.expand(u))))

The result is $u=-\frac{1}{2} x^2$. This function does not obey $u'(1)=0$ and
is completely wrong. In this least squares method we cannot access the basis
function $x$, which is needed in the exact solution, and we have no means
to obtain $u'(1)=0$.

**Remark.** There is a modification of the least squares method that
can be applied here. The
problem $u''=1$ must be rewritten as a system of two equations,
$u_1'=u_2$, $u_2' =1$. We expand $u_1=\sum_{j=0}^N c_j{\psi}_j$ and
$u_2=\sum_{j=0}^N d_j{\psi}_j$. The residuals in both equations are
added, squared, and differentiated with respect to $c_i$ and $d_i$,
$i=0,\ldots,N$. The result is a coupled equation system for the
$c_i$ and $d_i$ coefficients.

<!-- --- end solution of exercise --- -->

Filename: `cable_xn`.

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 4: Check integration by parts
<div id="fem:deq:exer:intg:parts"></div>

Consider the Galerkin method for the problem involving $u$
in [Exercise 2: Compute the deflection of a cable with sine functions](#fem:deq:exer:tension:cable).
Show that the formulas for $c_j$ are independent of whether we perform
integration by parts or not.


<!-- --- begin solution of exercise --- -->
**Solution.**
The Galerkin method is

$$
(u'',v)=(1,v)\quad\forall v\in V,
$$

and with the choice of $V$ we get

$$
\begin{align*}
A_{i,j} &=-(i+1)^2\pi^2 \int_0^1\sin^2((i+1)\frac{\pi x}{2})dx,\\ 
b_i &= \int_0^1\sin((i+1)\frac{\pi x}{2})dx
\end{align*}
$$

From [Exercise 2: Compute the deflection of a cable with sine functions](#fem:deq:exer:tension:cable) we realize that
the integrals are the same as in the least squares method, and
those results were identical to those of the Galerkin method with
integration by parts.

<!-- --- end solution of exercise --- -->
Filename: `cable_integr_by_parts`.

<!-- --- end exercise --- -->